In [1]:
from __future__ import print_function # Python 2/3 compatibility
import cv2 # Import the OpenCV library
import numpy as np # Import Numpy library
from scipy.spatial.transform import Rotation as R
import math # Math library
from numpy.linalg import inv

/home/peerajak/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/peerajak/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/peerajak/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/peerajak/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# Dictionary that was used to generate the ArUco marker
aruco_dictionary_name = "DICT_6X6_250"
 
# The different ArUco dictionaries built into the OpenCV library. 
ARUCO_DICT = {
  "DICT_4X4_50": cv2.aruco.DICT_4X4_50,
  "DICT_4X4_100": cv2.aruco.DICT_4X4_100,
  "DICT_4X4_250": cv2.aruco.DICT_4X4_250,
  "DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
  "DICT_5X5_50": cv2.aruco.DICT_5X5_50,
  "DICT_5X5_100": cv2.aruco.DICT_5X5_100,
  "DICT_5X5_250": cv2.aruco.DICT_5X5_250,
  "DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
  "DICT_6X6_50": cv2.aruco.DICT_6X6_50,
  "DICT_6X6_100": cv2.aruco.DICT_6X6_100,
  "DICT_6X6_250": cv2.aruco.DICT_6X6_250,
  "DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
  "DICT_7X7_50": cv2.aruco.DICT_7X7_50,
  "DICT_7X7_100": cv2.aruco.DICT_7X7_100,
  "DICT_7X7_250": cv2.aruco.DICT_7X7_250,
  "DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
  "DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL
}

In [3]:
# Side length of the ArUco marker in meters 
aruco_marker_side_length = 0.026

In [4]:
def euler_from_quaternion(x, y, z, w):
  """
  Convert a quaternion into euler angles (roll, pitch, yaw)
  roll is rotation around x in radians (counterclockwise)
  pitch is rotation around y in radians (counterclockwise)
  yaw is rotation around z in radians (counterclockwise)
  """
  t0 = +2.0 * (w * x + y * z)
  t1 = +1.0 - 2.0 * (x * x + y * y)
  roll_x = math.atan2(t0, t1)
      
  t2 = +2.0 * (w * y - z * x)
  t2 = +1.0 if t2 > +1.0 else t2
  t2 = -1.0 if t2 < -1.0 else t2
  pitch_y = math.asin(t2)
      
  t3 = +2.0 * (w * z + x * y)
  t4 = +1.0 - 2.0 * (y * y + z * z)
  yaw_z = math.atan2(t3, t4)
      
  return roll_x, pitch_y, yaw_z # in radians

In [5]:
def camera_coordinate_to_pixel_position(focal_x,focal_y,pixel_position_principal_point_x, pixel_position_principal_point_y, P_camra_coordinate):
    ox = round(pixel_position_principal_point_x)
    oy = round(pixel_position_principal_point_y)
    xc = P_camra_coordinate[0]
    yc = P_camra_coordinate[1]
    zc = P_camra_coordinate[2] 
    sx = 1
    sy = 1
    x_im = -(focal_x*xc)/(sx*zc) + ox
    y_im = -(focal_y*yc)/(sy*zc) + oy
    return x_im, y_im


In [6]:
def main():
  global p98
  global p98_r
  global p40
  global p40_r
    
  is_p98_defined = False
  is_p40_defined = False
  """
  Main method of the program.
  """
  # Check that we have a valid ArUco marker
  if ARUCO_DICT.get(aruco_dictionary_name, None) is None:
    print("[INFO] ArUCo tag of '{}' is not supported".format(
      args["type"]))
    sys.exit(0)
 
  # Load the camera parameters from the saved file
  #cv_file = cv2.FileStorage( camera_calibration_parameters_filename, cv2.FILE_STORAGE_READ) 
  #mtx = cv_file.getNode('K').mat()
  #dst = cv_file.getNode('D').mat()
  #cv_file.release()

  mtx_np = np.array([ [759.895784, 0.000000, 312.753105],[0.000000, 762.113647, 214.923553], [0., 0., 1.]], np.float32)
  mtx = mtx_np
  dst_np = np.array([0.062948, -0.273568, 0.005933, -0.001056, 0.000000], np.float32)   
  prj_np = np.array([[761.265137, 0.000000, 311.720175, 0.000000],\
                    [0.000000, 764.304443, 215.883204, 0.000000],\
                    [0.000000, 0.000000, 1.000000, 0.000000]], np.float32)   
  dst = dst_np
  # Load the ArUco dictionary
  print("[INFO] detecting '{}' markers...".format(
    aruco_dictionary_name))
  this_aruco_dictionary = cv2.aruco.Dictionary_get(ARUCO_DICT[aruco_dictionary_name])
  this_aruco_parameters = cv2.aruco.DetectorParameters_create()
   
  # Start the video stream
  cap = cv2.VideoCapture(0)

  if cap is None:
        print("failed to capture videos")
        return

   
  while(True):
      # Capture frame-by-frame
      # This method returns True/False as well
      # as the video frame.
      ret, streamImage = cap.read() 
      detectingImage = streamImage.copy() 

      # Detect ArUco markers in the video frame
      (corners, marker_ids, rejected) = cv2.aruco.detectMarkers(
          detectingImage , this_aruco_dictionary, parameters=this_aruco_parameters,
          cameraMatrix=mtx, distCoeff=dst)



      # Check that at least one ArUco marker was detected
      if marker_ids is not None: 
          # print('corners ',corners)
          # print('marker_ids', marker_ids)
          # Draw a square around detected markers in the video frame
          cv2.aruco.drawDetectedMarkers(detectingImage , corners, marker_ids)

          # Get the rotation and translation vectors
          rvecs, tvecs, obj_points = cv2.aruco.estimatePoseSingleMarkers(
            corners,
            aruco_marker_side_length,
            mtx,
            dst)
          # Print the pose for the ArUco marker
          # The pose of the marker is with respect to the camera lens frame.
          # Imagine you are looking through the camera viewfinder, 
          # the camera lens frame's:
          # x-axis points to the right
          # y-axis points straight down towards your toes
          # z-axis points straight ahead away from your eye, out of the camera
          for i, marker_id in enumerate(marker_ids):

            # Store the translation (i.e. position) information
            transform_translation_x = tvecs[i][0][0]
            transform_translation_y = tvecs[i][0][1]
            transform_translation_z = tvecs[i][0][2]

            # Store the rotation information
            rotation_matrix = np.eye(4)
            rotation_matrix[0:3, 0:3] = cv2.Rodrigues(np.array(rvecs[i][0]))[0]
            r = R.from_matrix(rotation_matrix[0:3, 0:3])
            quat = r.as_quat()   

            # Quaternion format     
            transform_rotation_x = quat[0] 
            transform_rotation_y = quat[1] 
            transform_rotation_z = quat[2] 
            transform_rotation_w = quat[3] 

            # Euler angle format in radians
            roll_x, pitch_y, yaw_z = euler_from_quaternion(transform_rotation_x, 
                                                           transform_rotation_y, 
                                                           transform_rotation_z, 
                                                           transform_rotation_w)

            # roll_x = math.degrees(roll_x)
            # pitch_y = math.degrees(pitch_y)
            # yaw_z = math.degrees(yaw_z)
            # print("transform_translation_x: {}".format(transform_translation_x))
            # print("transform_translation_y: {}".format(transform_translation_y))
            # print("transform_translation_z: {}".format(transform_translation_z))
            # print("roll_x: {}".format(roll_x))
            # print("pitch_y: {}".format(pitch_y))
            # print("yaw_z: {}".format(yaw_z))
            # print()
            (topLeft, topRight, bottomRight, bottomLeft) = corners[i].reshape((4, 2))
            # convert each of the (x, y)-coordinate pairs to integers
            topRight = (int(topRight[0]), int(topRight[1]))
            bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
            bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
            topLeft = (int(topLeft[0]), int(topLeft[1]))
            cX = int((topLeft[0] + bottomRight[0]) / 2.0)
            cY = int((topLeft[1] + bottomRight[1]) / 2.0)
            #print("id {} ".format(marker_id))
            #print("at image position ",[cX,cY])
            #print("rotation matrix",rotation_matrix[0:3, 0:3] )
            #print("translation matrix",[transform_translation_x ,transform_translation_y,transform_translation_z])
            #print(obj_points[i])
            if marker_id == 98:
                p98_im = np.array([cX,cY], dtype='>i4')
                p98 = tvecs[i][0][0:3]
                p98_r = rotation_matrix[0:3, 0:3]
                #print('set 98',p98, obj_points[i])
                is_p98_defined = True
            if marker_id == 40:
                p40_im = np.array([cX,cY], dtype='>i4')
                p40 = tvecs[i][0][0:3]
                p40_r = rotation_matrix[0:3, 0:3]  
                #print('set 40',p40, obj_points[i])
                is_p40_defined = True
            #print("============")
            if is_p98_defined and is_p40_defined:
                T_c_p98 = p98_r
                T_c_p40 = p40_r

                p40__p98 = np.dot(T_c_p98, p40 - p98  )
                #print(p40__p98)

                p98__p40 = np.dot(T_c_p40, p98 - p40  )
                #print(p98__p40)
                p40__p98_str = '40 in 98 frame: x={:3.2f}, y={:3.2f}, z={:3.2f} cm'.format(p40__p98[0]*100,p40__p98[1]*100,p40__p98[2]*100 )
                p98__p40_str = '98 in 40 frame: x={:3.2f}, y={:3.2f}, z={:3.2f} cm'.format(p98__p40[0]*100,p98__p40[1]*100,p98__p40[2]*100 )
                # Window name in which image is displayed
                window_name = 'Image'
                

                start_point = (p98_im[0], p98_im[1])
                end_point = (p40_im[0], p40_im[1])
                # Black color in BGR
                color = (0, 255, 255)
                # Line thickness of 5 px
                thickness = 1
                # Using cv2.line() method
                # Draw a diagonal black line with thickness of 5 px
                detectingImage = cv2.line(detectingImage, start_point, end_point, color, thickness)
                
                
                
                
                focal_x = mtx_np[0][0]
                focal_y = mtx_np[1][1]
                pixel_position_principal_point_x = mtx_np[0][2]
                pixel_position_principal_point_y = mtx_np[1][2]
                #x_40_im_f, y_40_im_f = camera_coordinate_to_pixel_position(focal_x,focal_y,pixel_position_principal_point_x, pixel_position_principal_point_y, p40)
                #x_40_im = int(round(x_40_im_f))
                #y_40_im = int(round(y_40_im_f))
                p40_homo = np.append(p40, 1.)
                p40_c = np.dot(prj_np, p40_homo)
                uv1_40 = (np.dot(mtx_np, p40_c)).flatten()
                x_40_im = int(round(uv1_40[0]))
                y_40_im = int(round(uv1_40[1]))                
                #print(x_40_im,y_40_im,p40_im, p40)
                detectingImage = cv2.circle(detectingImage, (x_40_im, y_40_im), radius=5, color=(0, 0, 255), thickness=5)
                
            
                
                
                # font
                font = cv2.FONT_HERSHEY_SIMPLEX
                org = (50, 50)
                fontScale = 0.5   
                color = (0, 255, 255)
                thickness = 1    
                detectingImage = cv2.putText(detectingImage, p98__p40_str , (50, 50), font,fontScale, color, thickness, cv2.LINE_AA)
                detectingImage = cv2.putText(detectingImage, p40__p98_str , (50, 80), font,fontScale, color, thickness, cv2.LINE_AA)

            # Draw the axes on the marker
            cv2.aruco.drawAxis(detectingImage , mtx, dst, rvecs[i], tvecs[i], 0.05)

      # Display the resulting frame

      cv2.imshow('frame',detectingImage )
          
      # If "q" is pressed on the keyboard, 
      # exit this loop
      if cv2.waitKey(1) & 0xFF == ord('q'):
          # filename_org = 'data/original_image.jpg'
          # filename_detected = 'data/detected_image.jpg'
          # cv2.imwrite(filename_org, streamImage)
          # cv2.imwrite(filename_detected, detectingImage)
          break


  
  # Close down the video stream
  cap.release()
  cv2.destroyAllWindows()


In [7]:
   
if __name__ == '__main__':
  print(__doc__)
  main()

Automatically created module for IPython interactive environment
[INFO] detecting 'DICT_6X6_250' markers...


[ WARN:0] global /home/peerajak/__Features__/opencv-4.5.2/modules/videoio/src/cap_gstreamer.cpp (1081) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [8]:
# This might be wrong or right. I was expecting z component of p40__p98 to be nearly zero
# but this is because the camera calibration was done on the laptop computer

T_c_p98 = p98_r
T_c_p40 = p40_r

p40__p98 = np.dot(T_c_p98, p40 - p98  )
print(p40__p98)

p98__p40 = np.dot(T_c_p40, p98 - p40  )
print(p98__p40)

[-0.04325461 -0.04274871 -0.00035997]
[0.04386097 0.04203054 0.00286218]
